In [30]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import seaborn as sns

In [31]:
df = pd.read_csv('../../../data/processed_data/full_df_positive_mental_provider.csv')
df.head()

,State FIPS Code,County FIPS Code,5-digit FIPS Code,State Abbreviation,Name,Release Year,County Ranked (Yes=1/No=0),Premature death raw value,Premature death numerator,Premature death denominator,...,April Average Precipitation,May Average Precipitation,June Average Precipitation,July Average Precipitation,August Average Precipitation,September Average Precipitation,October Average Precipitation,November Average Precipitation,December Average Precipitation,RUCC
0,1,1,1001,AL,Autauga County,2011,1.0,9967.4,675.0,NaN,...,4.19,1.93,3.39,4.38,1.75,4.46,0.98,4.63,3.96,2.0
1,1,3,1003,AL,Baldwin County,2011,1.0,8321.8,2219.0,NaN,...,1.28,1.68,3.15,8.73,2.05,9.89,0.15,3.22,2.78,3.0
2,1,15,1015,AL,Calhoun County,2011,1.0,11719.6,2106.0,NaN,...,5.30,1.65,4.53,3.83,0.93,6.85,0.90,5.00,4.86,3.0
3,1,43,1043,AL,Cullman County,2011,1.0,9564.7,1245.0,NaN,...,9.50,2.30,3.76,5.03,1.51,10.39,0.75,5.95,6.09,4.0
4,1,55,1055,AL,Etowah County,2011,1.0,11812.3,1905.0,NaN,...,6.53,2.77,4.79,4.81,1.53,10.14,0.61,5.69,6.24,3.0


In [32]:
identifying_fields = ['State Abbreviation', 'Name', 'Release Year']
with open("../Regression Analysis/factors.txt") as f:
     factors = f.read().splitlines()
with open("../Regression Analysis/outcomes.txt") as f:
     outcomes = f.read().splitlines()
df = df[identifying_fields + factors + outcomes]
df.head()

,State Abbreviation,Name,Release Year,High school graduation raw value,Unemployment raw value,Some college raw value,Ratio of population to mental health providers,Median household income raw value,Average Temperature,January Average Temperature,...,June Average Precipitation,July Average Precipitation,August Average Precipitation,September Average Precipitation,October Average Precipitation,November Average Precipitation,December Average Precipitation,RUCC,Poor mental health days raw value,Crude Rate
0,AL,Autauga County,2011,0.740,0.089,0.554,50354.0,51622.0,64.658333,41.5,...,3.39,4.38,1.75,4.46,0.98,4.63,3.96,2.0,4.1,21.7
1,AL,Baldwin County,2011,0.681,0.088,0.615,6076.0,51957.0,67.733333,47.6,...,3.15,8.73,2.05,9.89,0.15,3.22,2.78,3.0,4.1,18.2
2,AL,Calhoun County,2011,0.689,0.101,0.499,9452.0,39997.0,62.325000,39.1,...,4.53,3.83,0.93,6.85,0.90,5.00,4.86,3.0,4.8,14.4
3,AL,Cullman County,2011,0.684,0.094,0.460,20380.0,39276.0,61.333333,37.1,...,3.76,5.03,1.51,10.39,0.75,5.95,6.09,4.0,4.8,17.4
4,AL,Etowah County,2011,0.702,0.105,0.561,10344.0,37264.0,62.100000,38.7,...,4.79,4.81,1.53,10.14,0.61,5.69,6.24,3.0,4.4,24.0


In [33]:
locations = list(range(9)) + [21] + list(range(34, 37))
df.iloc[:, locations].columns

Index(['State Abbreviation', 'Name', 'Release Year',
       'High school graduation raw value', 'Unemployment raw value',
       'Some college raw value',
       'Ratio of population to mental health providers',
       'Median household income raw value', 'Average Temperature',
       'Average Precipitation', 'RUCC', 'Poor mental health days raw value',
       'Crude Rate'],
      dtype='object')

In [34]:
# Assign into df variables
df_small = df.iloc[:, locations]
df_small.head()

,State Abbreviation,Name,Release Year,High school graduation raw value,Unemployment raw value,Some college raw value,Ratio of population to mental health providers,Median household income raw value,Average Temperature,Average Precipitation,RUCC,Poor mental health days raw value,Crude Rate
0,AL,Autauga County,2011,0.740,0.089,0.554,50354.0,51622.0,64.658333,3.769167,2.0,4.1,21.7
1,AL,Baldwin County,2011,0.681,0.088,0.615,6076.0,51957.0,67.733333,3.720833,3.0,4.1,18.2
2,AL,Calhoun County,2011,0.689,0.101,0.499,9452.0,39997.0,62.325000,4.231667,3.0,4.8,14.4
3,AL,Cullman County,2011,0.684,0.094,0.460,20380.0,39276.0,61.333333,5.382500,4.0,4.8,17.4
4,AL,Etowah County,2011,0.702,0.105,0.561,10344.0,37264.0,62.100000,4.965000,3.0,4.4,24.0


In [35]:
df_small["RUCC"].describe()

count    4471.000000
mean        2.522926
std         1.527921
min         1.000000
25%         1.000000
50%         2.000000
75%         3.000000
max         9.000000
Name: RUCC, dtype: float64

In [36]:
df_small = df_small.dropna()

In [37]:
for col in df_small.columns[3:13]:
    print(col)
    print('Average: {}'.format(df_small[col].mean()))
    print('Median: {}'.format(df_small[col].median()))
    print()

High school graduation raw value
Average: 0.8086074917278316
Median: 0.82

Unemployment raw value
Average: 0.0815502352142085
Median: 0.0779643064

Some college raw value
Average: 0.6023396813312317
Median: 0.6010806075

Ratio of population to mental health providers
Average: 5262.059949512721
Median: 2092.2

Median household income raw value
Average: 51249.324926818284
Median: 48324.0

Average Temperature
Average: 56.46524806725212
Median: 55.76666666666665

Average Precipitation
Average: 3.523053741649778
Median: 3.503333333333334

RUCC
Average: 2.523305561810403
Median: 2.0

Poor mental health days raw value
Average: 3.5762215717180816
Median: 3.5

Crude Rate
Average: 17.094685881558206
Median: 15.6



In [40]:
df_small['Ratio of population to mental health providers'].describe()

count      4441.000000
mean       5262.059950
std        9562.788332
min          73.229508
25%         813.290323
50%        2092.200000
75%        5452.875000
max      106755.000000
Name: Ratio of population to mental health providers, dtype: float64

In [39]:
df_small[df_small['Ratio of population to mental health providers'] > 9000].count()

State Abbreviation                                658
Name                                              658
Release Year                                      658
High school graduation raw value                  658
Unemployment raw value                            658
Some college raw value                            658
Ratio of population to mental health providers    658
Median household income raw value                 658
Average Temperature                               658
Average Precipitation                             658
RUCC                                              658
Poor mental health days raw value                 658
Crude Rate                                        658
dtype: int64

In [41]:
# Create new column
def classify_treatment(row, treatment, mid, reverse):
    if treatment == 'Ratio of population to mental health providers':
        if reverse:
            return 0 if row[treatment] >= 9000 else 1
        else:
            return 1 if row[treatment] >= 9000 else 0
    if reverse:
        return 0 if row[treatment] >= mid else 1
    else:
        return 1 if row[treatment] >= mid else 0

def create_new_column(df, treatment, categorize_treatment, reverse=False):
    mid = (df[treatment].median() + df[treatment].mean()) / 2
    print('{} split = {}'.format(treatment, mid))
    df[categorize_treatment] = df.apply(classify_treatment, args=(treatment, mid, reverse), axis=1)

In [42]:
# try once
create_new_column(df_small, "High school graduation raw value", "high_school_treated")
create_new_column(df_small, "Unemployment raw value", "unemployment_treated", reverse=True)
create_new_column(df_small, "Some college raw value", "college_treated")
create_new_column(df_small, "Ratio of population to mental health providers", "mental_health_provider_treated", reverse=True)
create_new_column(df_small, "Median household income raw value", "household_income_treated")
create_new_column(df_small, "Average Temperature", "temperature_treated", reverse=True)
create_new_column(df_small, "Average Precipitation", "precipitation_treated", reverse=True)
create_new_column(df_small, "RUCC", "rucc_treated", reverse=True)
df_small.head()

High school graduation raw value split = 0.8143037458639157
Unemployment raw value split = 0.07975727080710425
Some college raw value split = 0.6017101444156159
Ratio of population to mental health providers split = 3677.1299747563603
Median household income raw value split = 49786.66246340914
Average Temperature split = 56.115957366959385
Average Precipitation split = 3.513193537491556
RUCC split = 2.2616527809052016


,State Abbreviation,Name,Release Year,High school graduation raw value,Unemployment raw value,Some college raw value,Ratio of population to mental health providers,Median household income raw value,Average Temperature,Average Precipitation,...,Poor mental health days raw value,Crude Rate,high_school_treated,unemployment_treated,college_treated,mental_health_provider_treated,household_income_treated,temperature_treated,precipitation_treated,rucc_treated
0,AL,Autauga County,2011,0.740,0.089,0.554,50354.0,51622.0,64.658333,3.769167,...,4.1,21.7,0,0,0,0,1,0,0,1
1,AL,Baldwin County,2011,0.681,0.088,0.615,6076.0,51957.0,67.733333,3.720833,...,4.1,18.2,0,0,1,1,1,0,0,0
2,AL,Calhoun County,2011,0.689,0.101,0.499,9452.0,39997.0,62.325000,4.231667,...,4.8,14.4,0,0,0,0,0,0,0,0
3,AL,Cullman County,2011,0.684,0.094,0.460,20380.0,39276.0,61.333333,5.382500,...,4.8,17.4,0,0,0,0,0,0,0,0
4,AL,Etowah County,2011,0.702,0.105,0.561,10344.0,37264.0,62.100000,4.965000,...,4.4,24.0,0,0,0,0,0,0,0,0


In [46]:
df_small

,State Abbreviation,Name,Release Year,High school graduation raw value,Unemployment raw value,Some college raw value,Ratio of population to mental health providers,Median household income raw value,Average Temperature,Average Precipitation,...,Poor mental health days raw value,Crude Rate,high_school_treated,unemployment_treated,college_treated,mental_health_provider_treated,household_income_treated,temperature_treated,precipitation_treated,rucc_treated
0,AL,Autauga County,2011,0.740000,0.089000,0.554000,50354.000000,51622.0,64.658333,3.769167,...,4.1,21.7,0,0,0,0,1,0,0,1
1,AL,Baldwin County,2011,0.681000,0.088000,0.615000,6076.000000,51957.0,67.733333,3.720833,...,4.1,18.2,0,0,1,1,1,0,0,0
2,AL,Calhoun County,2011,0.689000,0.101000,0.499000,9452.000000,39997.0,62.325000,4.231667,...,4.8,14.4,0,0,0,0,0,0,0,0
3,AL,Cullman County,2011,0.684000,0.094000,0.460000,20380.000000,39276.0,61.333333,5.382500,...,4.8,17.4,0,0,0,0,0,0,0,0
4,AL,Etowah County,2011,0.702000,0.105000,0.561000,10344.000000,37264.0,62.100000,4.965000,...,4.4,24.0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4466,WY,Albany County,2016,0.860000,0.035655,0.861182,207.752747,42838.0,42.650000,1.175000,...,3.7,26.1,1,1,1,1,0,1,1,0
4467,WY,Fremont County,2016,0.680985,0.051891,0.628718,550.040541,48624.0,42.658333,1.559167,...,3.5,47.2,0,1,1,1,0,1,1,0
4468,WY,Laramie County,2016,0.725947,0.044848,0.710952,299.344720,57192.0,47.758333,1.345833,...,3.2,25.5,0,1,1,1,1,1,1,0
4469,WY,Natrona County,2016,0.740000,0.042004,0.663984,316.372093,57427.0,45.625000,1.089167,...,3.3,21.0,0,1,1,1,1,1,1,0


In [56]:
df_small.groupby("rucc_treated").count()

,State Abbreviation,Name,Release Year,High school graduation raw value,Unemployment raw value,Some college raw value,Ratio of population to mental health providers,Median household income raw value,Average Temperature,Average Precipitation,RUCC,Poor mental health days raw value,Crude Rate,high_school_treated,unemployment_treated,college_treated,mental_health_provider_treated,household_income_treated,temperature_treated,precipitation_treated
rucc_treated,,,,,,,,,,,,,,,,,,,,
0,1897,1897,1897,1897,1897,1897,1897,1897,1897,1897,1897,1897,1897,1897,1897,1897,1897,1897,1897,1897
1,2544,2544,2544,2544,2544,2544,2544,2544,2544,2544,2544,2544,2544,2544,2544,2544,2544,2544,2544,2544


In [57]:
df_small.to_csv('../../../data/processed_data/df_matching.csv', index=False)

## Remove Null High School Value (examine if nonrandom missing)

In [19]:
df_small["high_school_missing"] = df_small["High school graduation raw value"].isnull()

/var/folders/52/9bx80g11757dncrhv6pqh9f40000gn/T/ipykernel_59913/2474497170.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_small["high_school_missing"] = df_small["High school graduation raw value"].isnull()


In [20]:
df_small.head()
df_non_miss = df_small.dropna(subset=['High school graduation raw value'])

In [21]:
def compute_smd(table_1: pd.DataFrame, table_2: pd.DataFrame, covariates: list, round_digits: int=4) -> pd.DataFrame:
    covariates_smds = []
    for var in covariates:
        table_2_values = table_2[var].values
        pos_mean = table_2_values[0]
        pos_std = table_2_values[1]
        
        table_1_values = table_1[var].values
        neg_mean = table_1_values[0]
        neg_std = table_1_values[0]

        smd = (pos_mean - neg_mean) / np.sqrt((pos_std ** 2 + neg_std ** 2) / 2)
        smd = round(smd, round_digits)
        covariates_smds.append(smd)

    return pd.DataFrame({'features': covariates, 'smd': covariates_smds})

In [22]:
covariates_list = df_small.columns[3:13]
agg_operations = {}
agg_operations.update({
    covariate: ['mean', 'std'] for covariate in covariates_list
})
all_table = df_small.agg(agg_operations)
non_miss_table = df_non_miss.agg(agg_operations)
all_table

,High school graduation raw value,Unemployment raw value,Some college raw value,Ratio of population to mental health providers,Median household income raw value,Average Temperature,Average Precipitation,RUCC,Poor mental health days raw value,Crude Rate
mean,0.808850,0.081516,0.602417,5261.542380,51307.074033,56.472357,3.526804,2.522926,3.574092,17.112279
std,0.091176,0.026890,0.099033,9547.357392,13092.222205,8.179814,1.387475,1.527921,0.658281,7.556027


In [23]:
non_miss_table

,High school graduation raw value,Unemployment raw value,Some college raw value,Ratio of population to mental health providers,Median household income raw value,Average Temperature,Average Precipitation,RUCC,Poor mental health days raw value,Crude Rate
mean,0.808850,0.081531,0.602252,5262.296438,51259.004270,56.481491,3.525641,2.525393,3.576222,17.123775
std,0.091176,0.026818,0.098968,9564.014718,13055.478618,8.184281,1.387077,1.529189,0.658032,7.557870


In [24]:
# standardize mean difference, <= 0.25
# Put both variables in there, unemployment/education balanced
covariates_smd = compute_smd(all_table, non_miss_table, covariates_list)
print(covariates_smd)

                                         features     smd
0                High school graduation raw value  0.0000
1                          Unemployment raw value  0.0002
2                          Some college raw value -0.0004
3  Ratio of population to mental health providers  0.0001
4               Median household income raw value -0.0013
5                             Average Temperature  0.0002
6                           Average Precipitation -0.0004
7                                            RUCC  0.0012
8               Poor mental health days raw value  0.0008
9                                      Crude Rate  0.0009


In [25]:
df_non_miss = df_non_miss.iloc[:,:-1] # get rid of last col

In [26]:
df_non_miss = df_non_miss.dropna()
df_non_miss

,State Abbreviation,Name,Release Year,High school graduation raw value,Unemployment raw value,Some college raw value,Ratio of population to mental health providers,Median household income raw value,Average Temperature,Average Precipitation,...,Poor mental health days raw value,Crude Rate,high_school_treated,unemployment_treated,college_treated,mental_health_provider_treated,household_income_treated,temperature_treated,precipitation_treated,rucc_treated
0,AL,Autauga County,2011,0.740000,0.089000,0.554000,50354.000000,51622.0,64.658333,3.769167,...,4.1,21.7,0,0,0,0,1,0,0,1
1,AL,Baldwin County,2011,0.681000,0.088000,0.615000,6076.000000,51957.0,67.733333,3.720833,...,4.1,18.2,0,0,1,1,1,0,0,0
2,AL,Calhoun County,2011,0.689000,0.101000,0.499000,9452.000000,39997.0,62.325000,4.231667,...,4.8,14.4,0,0,0,0,0,0,0,0
3,AL,Cullman County,2011,0.684000,0.094000,0.460000,20380.000000,39276.0,61.333333,5.382500,...,4.8,17.4,0,0,0,0,0,0,0,0
4,AL,Etowah County,2011,0.702000,0.105000,0.561000,10344.000000,37264.0,62.100000,4.965000,...,4.4,24.0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4466,WY,Albany County,2016,0.860000,0.035655,0.861182,207.752747,42838.0,42.650000,1.175000,...,3.7,26.1,1,1,1,1,0,1,1,0
4467,WY,Fremont County,2016,0.680985,0.051891,0.628718,550.040541,48624.0,42.658333,1.559167,...,3.5,47.2,0,1,1,1,0,1,1,0
4468,WY,Laramie County,2016,0.725947,0.044848,0.710952,299.344720,57192.0,47.758333,1.345833,...,3.2,25.5,0,1,1,1,1,1,1,0
4469,WY,Natrona County,2016,0.740000,0.042004,0.663984,316.372093,57427.0,45.625000,1.089167,...,3.3,21.0,0,1,1,1,1,1,1,0
